In [1]:
import requests, json, pandas as pd
from collections import defaultdict
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: No module named bokeh.plotting

In [7]:
statewise_uri = "https://api.covid19india.org/v3/timeseries.json"

def getTimeSeriesDataStateWise():
    req = requests.get(statewise_uri, verify=False)
    return req.json()

In [8]:
def createTableFromJson(jsondata):
    # col = ['date', 'state','delta_confirmed', 'delta_recovered', 'delta_tested',
    #        'total_confirmed', 'total_deceased', 'total_recovered', 'total_tested']

    d = defaultdict(list)

    for state, data in jsondata.items():
        for date, v in data.items():
            d['state'].append(state)
            d['date'].append(date)
            delta = v.get('delta', {})
            d['delta_confirmed'].append(delta.get('confirmed', 0))
            d['delta_deceased'].append(delta.get('deceased', 0))
            d['delta_recovered'].append(delta.get('recovered', 0))
            d['delta_tested'].append(delta.get('tested', 0))
            total = v.get('total', {})
            d['total_confirmed'].append(total.get('confirmed', 0))
            d['total_deceased'].append(total.get('deceased', 0))
            d['total_recovered'].append(total.get('recovered', 0))
            d['total_tested'].append(total.get('tested', 0))

    df = pd.DataFrame.from_dict(d)
    return df

In [9]:

def cleanData(df):
    mask_curr_date = (df['date'] > '2020-06-26') & (df['state'] != 'TT')
    new_df = df.loc[mask_curr_date].sort_values(by=['total_confirmed'], ascending=False)
    top_states = []
    for state in new_df[['state']].head(5).values.tolist():
        top_states.append(state[0])
    print(top_states)

    basic_mask = (df['date'] > '2020-04-15') & (df['state'] != 'TT') & (df['state'] != 'UN')
    top_states_mask, remaining_states_mask = False, basic_mask
    for state in top_states:
        top_states_mask = top_states_mask | (df['state'] == state)
        remaining_states_mask = remaining_states_mask & (df['state'] != state)

    top_states_mask = basic_mask & top_states_mask
    df['delta_confirmed_sma'] = df.iloc[:, 2].rolling(window=3).mean()
    top_state_df = pd.pivot_table(df.loc[top_states_mask], values='delta_confirmed_sma', index=['state'], 
                                  columns='date')
    rem_state_df = pd.pivot_table(df.loc[top_states_mask], values='delta_confirmed_sma', index=['state'], 
                                  columns='date')
    return (top_state_df, rem_state_df)

In [10]:
with open('raw_data/timeseries.json') as file:
    sj = json.load(file)
df = createTableFromJson(sj)
df.head()

,date,delta_confirmed,delta_deceased,delta_recovered,delta_tested,state,total_confirmed,total_deceased,total_recovered,total_tested
0,2020-05-25,0,0,0,407,DN,2,0,1,10085
1,2020-05-24,0,0,0,380,DN,2,0,1,9678
2,2020-05-27,0,0,0,484,DN,2,0,1,10872
3,2020-05-26,0,0,0,303,DN,2,0,1,10388
4,2020-05-21,0,0,0,319,DN,1,0,1,8456


In [11]:
top_state_df, rem_state_df = cleanData(df)

[u'MH', u'DL', u'TN', u'GJ', u'UP']


In [12]:
top_state_df.head()

date,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27
state,,,,,,,,,,,,,,,,,,,,,
DL,4.000000,3.666667,3.000000,2.666667,1.666667,1.333333,32.000000,5.333333,1.000000,1.333333,...,43.666667,22.000000,34.000000,53.000000,66.000000,23.666667,57.000000,44.000000,42.333333,43.000000
GJ,3.000000,3.000000,15.666667,11.666667,10.000000,12.000000,18.333333,14.666667,14.666667,10.666667,...,25.000000,23.000000,24.333333,26.333333,22.000000,9.666667,27.000000,15.000000,14.333333,12.000000
MH,9.000000,12.333333,32.666667,29.333333,13.666667,14.000000,73.000000,39.000000,21.333333,22.333333,...,80.666667,56.333333,73.666667,113.333333,130.000000,91.666667,153.333333,147.333333,127.666667,114.000000
TN,0.666667,0.666667,1.000000,1.000000,1.333333,1.666667,7.000000,4.666667,1.000000,1.000000,...,30.000000,13.666667,15.333333,31.666667,42.666667,13.000000,27.333333,28.000000,26.333333,38.000000
UP,1.000000,0.666667,2.666667,2.666667,1.333333,2.333333,6.333333,6.000000,1.666667,1.333333,...,14.000000,6.333333,13.000000,17.000000,20.666667,6.666667,13.000000,11.333333,9.000000,12.666667


In [13]:
rem_state_df.head()

date,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,...,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27
state,,,,,,,,,,,,,,,,,,,,,
DL,4.000000,3.666667,3.000000,2.666667,1.666667,1.333333,32.000000,5.333333,1.000000,1.333333,...,43.666667,22.000000,34.000000,53.000000,66.000000,23.666667,57.000000,44.000000,42.333333,43.000000
GJ,3.000000,3.000000,15.666667,11.666667,10.000000,12.000000,18.333333,14.666667,14.666667,10.666667,...,25.000000,23.000000,24.333333,26.333333,22.000000,9.666667,27.000000,15.000000,14.333333,12.000000
MH,9.000000,12.333333,32.666667,29.333333,13.666667,14.000000,73.000000,39.000000,21.333333,22.333333,...,80.666667,56.333333,73.666667,113.333333,130.000000,91.666667,153.333333,147.333333,127.666667,114.000000
TN,0.666667,0.666667,1.000000,1.000000,1.333333,1.666667,7.000000,4.666667,1.000000,1.000000,...,30.000000,13.666667,15.333333,31.666667,42.666667,13.000000,27.333333,28.000000,26.333333,38.000000
UP,1.000000,0.666667,2.666667,2.666667,1.333333,2.333333,6.333333,6.000000,1.666667,1.333333,...,14.000000,6.333333,13.000000,17.000000,20.666667,6.666667,13.000000,11.333333,9.000000,12.666667


In [ ]:
for mask in [top_states_mask, remaining_states_mask]:
        heatmap_table = pd.pivot_table(df.loc[mask], values='delta_confirmed_sma', index=['state'], columns='date')
        ax = sns.heatmap(heatmap_table, xticklabels=True, yticklabels=True, linewidths=.2, cmap='YlGn')
        plt.yticks(fontsize=6)
        plt.xticks(fontsize=6, rotation=45)
        plt.show(ax)